Загрузим датасет и основные компоненты

In [1]:
from surprise import accuracy, Dataset, SVDpp, SVD
import pandas as pd
import numpy as np
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
dataset = pd.DataFrame(data.raw_ratings, columns = ['iid', 'uid', 'rating', 'timestamp'])

trainset, testset = train_test_split(data, test_size=0.2)

Далее создадим гибридную рекомендательную систему на основе блендинга (взвешивания) двух моделей SVD и SVD++

In [2]:
pp = SVDpp()
pp.fit(trainset)
test_pred = pp.test(testset)
print(accuracy.rmse(test_pred, verbose=True))

RMSE: 0.9182
0.9181634920064818


In [3]:
svd = SVD()
svd.fit(trainset)    
test_pred = svd.test(testset)
print(accuracy.rmse(test_pred, verbose=True))

RMSE: 0.9396
0.939577289292495


Определим идентификатор фильма "Бойцовский клуб 1999г"

In [4]:
movies = pd.read_csv('data/movies.csv')
movieID = int(movies[movies.title == 'Fight Club (1999)'].movieId)

Выведем гибридные результаты. Сумму рейтингов умноженную на вес модели. Вес 0.6 для SVD++ и 0.4 для SVD.

In [5]:
print(pp.predict(294, movieID).est*0.6+svd.predict(294, movieID).est*0.4)

3.5298375


Далее выведем результаты подбора рекомендаций для обеих моделей

In [6]:
def generate_recommendation(uid, model, dataset, thresh=4.0, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)
    
    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
       
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))
            
            if len(rec_list) >= amount:
                return rec_list
            
            
print(generate_recommendation(294, svd, dataset))         
print(generate_recommendation(294, pp, dataset))   

[('474', 4.31), ('14', 4.07), ('198', 4.13), ('83', 4.17), ('211', 4.07)]
[('180', 4.05), ('60', 4.01), ('611', 4.03), ('474', 4.19), ('98', 4.18)]


In [ ]:
В списках видно, что рекомендации отличаются, можно выбрать наилучшие рейтинги